## Back Test Data Generation

In [1]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
pd.get_option("display.max_columns", None)

20

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                            dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                            dataset.iloc[indices.flatten()[i]]['col_1'],
                            dataset.iloc[indices.flatten()[i]]['col_2'],
                            dataset.iloc[indices.flatten()[i]]['col_3'],
#                            dataset.iloc[indices.flatten()[i]]['col_4'],
#                            dataset.iloc[indices.flatten()[i]]['col_5'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_5'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_10'],
#                            dataset.iloc[indices.flatten()[i]]['F_SMA_20']
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1',
                      'col_2',
                      'col_3',
#                      'col_4',
#                      'col_5',
                      'F_SMA_5',
                      'F_SMA_10',
#                      'F_SMA_20',
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
filename = 'EUR_USD_H4.csv'
data = pd.read_csv(filename)

In [6]:
data.columns

Index(['Date', 'Time', 'f_time', 'julian_date', 'Weekday', 'Weekday_Name',
       'UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close', 'SMA_5', 'SMA_10',
       'F_SMA_5', 'F_SMA_10', 'col_1', 'col_2', 'col_3'],
      dtype='object')

In [7]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,F_SMA_5,F_SMA_10,col_1,col_2,col_3
0,2015-09-07,01:00:00,1,250,0,Monday,2015-09-07T01:00:00.000000000Z,3223,1.11666,1.11694,1.11412,1.11424,1.11460,1.11371,-0.00036,0.00053,0.00242,0.00270,0.00254
1,2015-09-07,05:00:00,5,250,0,Monday,2015-09-07T05:00:00.000000000Z,6486,1.11421,1.11647,1.11216,1.11625,1.11528,1.11416,0.00097,0.00209,-0.00204,0.00022,0.00205
2,2015-09-07,09:00:00,9,250,0,Monday,2015-09-07T09:00:00.000000000Z,5447,1.11623,1.11777,1.11388,1.11480,1.11536,1.11441,-0.00056,0.00039,0.00143,0.00297,0.00235
3,2015-09-07,13:00:00,13,250,0,Monday,2015-09-07T13:00:00.000000000Z,3012,1.11478,1.11732,1.11474,1.11692,1.11577,1.11482,0.00115,0.00210,-0.00214,0.00040,0.00004
4,2015-09-07,17:00:00,17,250,0,Monday,2015-09-07T17:00:00.000000000Z,1064,1.11692,1.11724,1.11641,1.11704,1.11585,1.11521,0.00119,0.00183,-0.00012,0.00020,0.00051


## Selecting 1000 Random Candles

In [8]:
Test_Candle = np.random.randint(low=1, high=len(data)-40, size=3000)
Test_Candle

array([2981, 3717, 7666, ..., 3352, 9549, 6359])

In [9]:
data.shape

(9649, 19)

In [10]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,F_SMA_5,F_SMA_10,col_1,col_2,col_3
0,2015-09-07,01:00:00,1,250,0,Monday,2015-09-07T01:00:00.000000000Z,3223,1.11666,1.11694,1.11412,1.11424,1.11460,1.11371,-0.00036,0.00053,0.00242,0.00270,0.00254
1,2015-09-07,05:00:00,5,250,0,Monday,2015-09-07T05:00:00.000000000Z,6486,1.11421,1.11647,1.11216,1.11625,1.11528,1.11416,0.00097,0.00209,-0.00204,0.00022,0.00205


# <font color='red'>CANDLE LOOP</font>

In [11]:
%%time

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              'Current_Market_Stoploss':[],
                              'Rec1':[],
                              'Rec1_P':[],
                              'Rec2':[],
                              'Rec2_P':[],
                              'Rec3':[],
                              'Rec3_P':[]
                             })

for candle_no in Test_Candle:

    #print('candle number is = ',candle_no)
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+5]
    #print('data is = ', data)
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market='Bullish'
        Current_Market_Stoploss= 0
    else:
        Current_Market = 'Bearish'
        Current_Market_Stoploss= 0 
    
    data = pd.read_csv(filename)
    data = data[['col_1','col_2','col_3','F_SMA_5','F_SMA_10']]

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    recs = []
    for indice in indices[1:4]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+5]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred).round(2)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade= 'BUY'
        else:
            Predicted_Trade = 'SELL'
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])
        
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              'Current_Market_Stoploss':Current_Market_Stoploss,
              'Rec1': predicted_output[0][0],
              'Rec1_P': predicted_output[0][1],
              'Rec2': predicted_output[1][0],
              'Rec2_P': predicted_output[1][1],
              'Rec3': predicted_output[2][0],
              'Rec3_P': predicted_output[2][1],
             }
    
    result_output = result_output.append(result, ignore_index = True)

CPU times: user 6min 2s, sys: 57.8 s, total: 7min
Wall time: 7min


In [12]:
result_output.to_csv('Back_Test_Result.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Result.csv')

In [13]:
result_output.head(50)

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,2981.0,35.0,Bearish,0.0,2.0,SELL,77.0,BUY,10.0,BUY
1,3717.0,10.0,Bearish,0.0,40.0,SELL,8.0,SELL,38.0,BUY
2,7666.0,65.0,Bullish,0.0,11.0,BUY,83.0,BUY,59.0,SELL
3,4097.0,63.0,Bearish,0.0,82.0,BUY,66.0,BUY,69.0,SELL
4,8700.0,98.0,Bullish,0.0,80.0,BUY,46.0,BUY,76.0,BUY
5,3517.0,22.0,Bullish,0.0,77.0,SELL,87.0,SELL,0.0,BUY
6,3279.0,90.0,Bearish,0.0,89.0,BUY,38.0,BUY,6.0,BUY
7,1778.0,11.0,Bearish,0.0,72.0,BUY,19.0,SELL,5.0,BUY
8,5059.0,54.0,Bullish,0.0,93.0,SELL,83.0,SELL,59.0,SELL
9,8120.0,45.0,Bearish,0.0,75.0,SELL,1.0,BUY,68.0,BUY
